In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import copy
import pickle
import sys
import time
sys.path.append("../")
from BayesCard.Evaluation.utils import parse_query
from Schemas.stats.schema import gen_stats_light_schema
from Join_scheme.data_prepare import identify_key_values, process_stats_data

In [2]:
data_path = "/Users/ziniuw/Desktop/research/Learned_QO/data/stats_simplified/{}.csv"
schema = gen_stats_light_schema(data_path)
all_keys, equivalent_keys = identify_key_values(schema)
print(equivalent_keys)

{'posts.Id': {'postLinks.RelatedPostId', 'posts.Id', 'tags.ExcerptPostId', 'postLinks.PostId', 'postHistory.PostId', 'votes.PostId', 'comments.PostId'}, 'users.Id': {'postHistory.UserId', 'votes.UserId', 'comments.UserId', 'posts.OwnerUserId', 'badges.UserId', 'users.Id'}}


In [3]:
def read_table_csv(table_obj, csv_seperator=',', stats=True):
    """
    Reads csv from path, renames columns and drops unnecessary columns
    """
    if stats:
        df_rows = pd.read_csv(table_obj.csv_file_location)
    else:
        df_rows = pd.read_csv(table_obj.csv_file_location, header=None, escapechar='\\', encoding='utf-8',
                              quotechar='"',
                              sep=csv_seperator)
    df_rows.columns = [table_obj.table_name + '.' + attr for attr in table_obj.attributes]

    for attribute in table_obj.irrelevant_attributes:
        df_rows = df_rows.drop(table_obj.table_name + '.' + attribute, axis=1)

    return df_rows.apply(pd.to_numeric, errors="ignore")

def get_data(data_path):
    if not data_path.endswith(".csv"):
        data_path += "/{}.csv"
    schema = gen_stats_light_schema(data_path)
    data = dict()
    for table_obj in schema.tables:
        table_name = table_obj.table_name
        df_rows = read_table_csv(table_obj, stats=True)
        data[table_name] = df_rows
        print(table_name)
    return data

all_data = get_data(data_path)

badges
votes
postHistory
posts
users
comments
postLinks
tags


In [4]:
from Evaluation.training import train_one_stats, test_trained_BN_on_stats
data_path = "/Users/ziniuw/Desktop/research/Learned_QO/data/stats_simplified/{}.csv"
model_path = "/Users/ziniuw/Desktop/research/Learned_QO/data/saved_models/"
train_one_stats("stats", data_path, model_path, 2, 200, "sub_optimal", True, actual_data = None)

bucketizing equivalent key group: {'postLinks.RelatedPostId', 'posts.Id', 'tags.ExcerptPostId', 'postLinks.PostId', 'postHistory.PostId', 'votes.PostId', 'comments.PostId'}
postLinks.RelatedPostId [ 7  4 10 11  6  7  9 11 21  7  9 19 19 10 11 14 13 19 17 22 16 11 15 18
 15 18 15 26 15 19 27 37 20 24 24 22 25 57 20 55 39 31 37 93 30 96 59 11]
posts.Id [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
tags.ExcerptPostId [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
postLinks.PostId [10  7  8  7  9  7  8  5 10  6  5  4  6  4 13  8 10  4  3  7  4  3  8  4
  2  4  4  3  4  3  5  4  6  5  5  5  5  3  4  2  1  5  6  3  2  2  4  0]
postHistory.PostId [18 17 23 27 22 21 30 20 19 24 19 21 18 21 23 19 17 18 18 29 16 15 18 21
 30 14 19 15 14 19 14 15 20 24 13 10  8 15 10 10 11 32 14 11 13 22 16  9]
vote

In [5]:
train_one_stats("stats", "", model_path+"update/", 2, 200, "sub_optimal", True, actual_data = all_data)

bucketizing equivalent key group: {'postLinks.RelatedPostId', 'posts.Id', 'tags.ExcerptPostId', 'postLinks.PostId', 'postHistory.PostId', 'votes.PostId', 'comments.PostId'}
postLinks.RelatedPostId [ 7  4 10 11  6  7  9 11 21  7  9 19 19 10 11 14 13 19 17 22 16 11 15 18
 15 18 15 26 15 19 27 37 20 24 24 22 25 57 20 55 39 31 37 93 30 96 59 11]
posts.Id [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
tags.ExcerptPostId [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
postLinks.PostId [10  7  8  7  9  7  8  5 10  6  5  4  6  4 13  8 10  4  3  7  4  3  8  4
  2  4  4  3  4  3  5  4  6  5  5  5  5  3  4  2  1  5  6  3  2  2  4  0]
postHistory.PostId [18 17 23 27 22 21 30 20 19 24 19 21 18 21 23 19 17 18 18 29 16 15 18 21
 30 14 19 15 14 19 14 15 20 24 13 10  8 15 10 10 11 32 14 11 13 22 16  9]
vote

In [ ]:
model_path = "/Users/ziniuw/Desktop/research/Learned_QO/data/saved_models/update/model_stats_sub_optimal_200.pkl"
with open(model_path, "rb") as f:
    BE2 = pickle.load(f)

In [ ]:
query57 = "SELECT COUNT(*) FROM posts as p, postLinks as pl, postHistory as ph, votes as v, badges as b, users as u WHERE p.Id = pl.RelatedPostId AND u.Id = p.OwnerUserId AND u.Id = b.UserId AND u.Id = ph.UserId AND u.Id = v.UserId AND p.CommentCount>=0 AND p.CommentCount<=13 AND ph.PostHistoryTypeId=5 AND ph.CreationDate<='2014-08-13 09:20:10'::timestamp AND v.CreationDate>='2010-07-19 00:00:00'::timestamp AND b.Date<='2014-09-09 10:24:35'::timestamp AND u.Views>=0 AND u.DownVotes>=0 AND u.CreationDate>='2010-08-04 16:59:53'::timestamp AND u.CreationDate<='2014-07-22 15:15:22'::timestamp;"
t = time.time()
res = BE2.get_cardinality_bound_one(query57)
print(time.time() - t)
print(res)

In [ ]:
for table in BE.bns:
    print(BE.bns[table].nrows)
    test_trained_BN_on_stats(BE.bns[table], table)

In [ ]:
model_path = "/Users/ziniuw/Desktop/research/Learned_QO/data/saved_models/update/model_stats_sub_optimal_200.pkl"
with open(model_path, "rb") as f:
    BE = pickle.load(f)

In [ ]:
query_file = "/Users/ziniuw/Desktop/research/Learned_QO/data/stats_simplified/stats_CEB_sub_queries.sql"
with open(query_file, "r") as f:
    queries = f.readlines()


In [ ]:
qerror = []
latency = []
pred = []
for i, query_str in enumerate(queries):
    query = query_str.split("||")[0][:-1]
    print("========================")
    true_card = int(query_str.split("||")[-1])
    t = time.time()
    res = BE.get_cardinality_bound_one(query)
    pred.append(res)
    latency.append(time.time() - t)
    qerror.append(res/true_card)
    print(f"estimating query {i}: predicted {res}, true_card {true_card}, qerror {res/true_card}, latency {time.time() - t}")

In [ ]:
qerror = np.asarray(qerror)

In [ ]:
temp_qerror = copy.deepcopy(qerror)
temp_qerror[temp_qerror < 1] = 1/temp_qerror[temp_qerror < 1]

In [ ]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(temp_qerror, i)}")
print(f"total inference time: {np.sum(latency)}")

In [ ]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(temp_qerror, i)}")
print(f"total inference time: {np.sum(latency)}")

In [ ]:
with open("stats_CEB_CE_scheme_200_greedy.txt", "w") as f:
    for p in pred:
        f.write(str(p)+"\n")

In [ ]:
with open("stats_CEB_exec.sql", "r") as f:
    queries = f.readlines()

In [ ]:
with open("stats_CEB_exec.sql", "w") as f:
    for q in queries:
        q = q.split("||")[-1]
        f.write(q)